In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math
import numpy as np
import random
time_e6 = np.loadtxt('time_edge6_15-23.csv')#time 
temp = time_e6.shape

edge_n = {}
with open('edgelist_6_15-23.txt') as file: 
    for line in file:
        if line[0] != '#' :
            edge1,edge2,number=line.split('|')
            number = number.strip('\n')
            edge_n[number] = (edge1,edge2)
time = temp[0]
edges = temp[1]

def garthgraph(ga):
    G = nx.Graph()
    tempe = 0
    for i in range(len(ga)):#边数的编号
        if ga[i] == 1 :
            tempe = edge_n[str(i)]
            #print(tempe[0],tempe[1])
            G.add_edge(tempe[0],tempe[1])
    return G

TCo

In [ ]:
def tcore(G): 
    G_c = G.copy()
    count = 0
    core_numb = nx.core_number(G_c)
    kmax = max(core_numb.values())

    Gcore = nx.k_core(G_c,kmax,core_numb)

    Gclist ={}
    for e in Gcore.edges():
        if Gcore.degree(e[0]) > kmax and Gcore.degree(e[1]) >kmax:
            Gclist[e] = 1
    def neiborcheck(G,k):
        flag = 0
        for n in nx.neighbors(G,k):
            if core_numb[n] > core_numb[k]:
                flag == 1
        return flag
                

    Gplist = {}
    onv = nx.onion_layers(G_c)
    for k in range(kmax):
        kshell = nx.k_shell(G_c,k,core_numb)
        for e in kshell.edges():
            if onv[e[0]] == onv[e[1]]:
                Gplist[e]=1
    
    for i in Gclist:
        
        x = i[0]
        y = i[1]
        for j in Gplist:
            a = j[0]
            b = j[1]
            #if neiborcheck(G_c,b) == 0 and neiborcheck(G_c,a) == 0 :
            if (x,a) not in G_c.edges() and (y,b) not in G_c.edges() and Gclist[i] == 1 and Gplist[j] == 1:
                Gcore = nx.k_core(G_c,kmax,core_numb)
                if Gcore.degree[x] > kmax and Gcore.degree[y] > kmax :
                    G_c.remove_edge(x,y)
                    G_c.remove_edge(a,b)
                    G_c.add_edge(x,a)
                    G_c.add_edge(y,b)
                    count += 1
                    Gclist[i] = 0
                    Gplist[j] = 0

                break
            elif (x,b) not in G_c.edges() and (y,a) not in G_c.edges() and Gclist[i] == 1 and Gplist[j] == 1:
                Gcore = nx.k_core(G_c,kmax,core_numb)
            
                if Gcore.degree[x] > kmax and Gcore.degree[y] > kmax :
                    G_c.remove_edge(x,y)
                    G_c.remove_edge(a,b)
                    G_c.add_edge(x,b)
                    G_c.add_edge(y,a)
                    count += 1
                    Gclist[i] = 0
                    Gplist[j] = 0
                break
    
       

    return G_c

DCo

In [ ]:

def dcore(G):
    G_c = G.copy()
    core_numb = nx.core_number(G_c)
    kmax = max(core_numb.values())

    Gcore = nx.k_core(G_c,kmax,core_numb)

    on = nx.onion_layers(Gcore)

    layer = {}

    for i in on:
        layer[on[i]] = []

    for i in on:
        layer[on[i]].append(i)

    Gts = []
    for w in layer:
        
        S = max(layer.keys())+1 -w
        #print(w,max(layer.keys()))
        for j in layer[S]:
            Gts.append(j)
        Gs = nx.subgraph(G_c,Gts)
        limt = min(kmax-1,len(Gts))
        Gtlist = {}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

        for i in range(len(Gts)):
            for j in range(i,len(Gts)):
                if (Gts[i],Gts[j]) not in Gs.nodes and i != j:
                    Gtlist[(Gts[i],Gts[j])] = 1
        time = 0
        for l in Gtlist:
            
            n1 = l[0]
            n2 = l[1]
            if  Gtlist[l] == 1 and Gs.degree(n1) <limt and Gs.degree(n2) <limt and n1 != n2:
                for i in nx.neighbors(G,n1):
                    if i not in Gts:
                        for j in nx.neighbors(G,n2  ):
                            if j not in Gts:
                                if (n1,i) in G_c.edges() and (n2,j) in G_c.edges():
                                    if i != j and core_numb[i] == core_numb[j] and (i,j) not in G_c.edges() and (n1,n2) not in G_c.edges and  Gtlist[l] == 1 :
                                        Gs = nx.subgraph(G_c,Gts)
                                        if Gs.degree(n1) <limt and Gs.degree(n2) <limt and n1!=n2 and i!=j:
                                            G_c.remove_edge(n1,i)
                                            G_c.remove_edge(n2,j)
                                            G_c.add_edge(n1,n2)
                                            G_c.add_edge(i,j)
                                            Gtlist[l] = 0
                                            if len([i for i in nx.connected_components(G_c)]) >1:
                                                G_c.remove_edge(n1,n2)
                                                G_c.remove_edge(i,j)
                                                G_c.add_edge(n1,i)
                                                G_c.add_edge(n2,j)
                                                Gtlist[l] = 1
                                            break
                        if Gtlist[l] == 0:
                            break
    return G_c

In [ ]:
def rcc(G):
    n = len(G.nodes)
    m = len(G.edges())
    return m/((n-1)*n/2)
def ccfd(ans):
    temp = {}
    for i in ans:
        temp[i] = 0
    for i in ans:
        for j in temp:
            if j >= i:
                temp[j] += ans[i]

    for i in ans:
        if i ==0:
            temp[0] = 1
        else:
            temp[i] /= i 

    return temp
def delnode(G0,dellist,nlist,ns):
    ans = {}
    G = G0.copy()
    Gsize = len(G.nodes)
    #print('delet len',len(dellist),ns)
    
    temp = 1
    for i in range(ns):
        if dellist[i] in G.nodes:
            
            G.remove_node(dellist[i])
            safter = len(max(nx.connected_components(G), key=len))
            ans[temp] = safter/Gsize
            # cm = len([i for i in nx.connected_components(G)])
            # ak = sum([len(i) for i in nx.connected_components(G)])
            # ans[temp] = ak/cm
            temp += 1
    tsa = ccfd(ans)    
    ans1 = {}
    for i in nlist:
        ans1[i] = tsa[i]   
    return ans1

def randomn(G,t):
    nodelist = G.nodes
    dellist = np.random.permutation(nodelist)
    return dellist[0:t]

def degreen(G,t):
    dlist = {}
    for i in G.nodes:
        dlist[i] = G.degree(i)
    drank = sorted(dlist.items(), key=lambda x: x[1], reverse=True)
    
    dellist = []
    for i in drank:
        dellist.append(i[0])
    return dellist[0:t]

In [ ]:
G = nx.Graph()
G = garthgraph(time_e6[0,:])
G_TCo = nx.Graph()
G_TCo = tcore(G)
G_DCo = nx.Graph()
G_DCo= dcore(G)

In [ ]:
nx.average_shortest_path_length(G)
nx.diameter(G)